# How to use RUPsycho

Welcome to the introductory guide for the RUPsycho Python package. This guide aims to provide a comprehensive overview of RUPsycho, a new and innovative software tool developed for the application of large language models (LLMs) in social science research. The RUPsycho package is designed to facilitate the exploration of humanlike behaviors through LLMs, offering a unique perspective in the field of natural language processing (NLP).

This guide will walk you through the various components and functionalities of RUPsycho. You will learn how to set up the package, navigate its testing environments, and effectively utilize its features for analyzing LLMs in the context of social science. The guide aims to equip researchers with the knowledge and skills required to leverage RUPsycho in their studies, contributing to a deeper understanding of the complex interplay between artificial intelligence and human psychology.

In [1]:
import rupsycho as rup

/home/jschelb/.pyenv/versions/3.10.8/envs/rup/lib/python3.10/site-packages/torch/cuda/__init__.py:611: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


## Load Experiments

### Load minimal Example

In [2]:
# experiment = rup.example_experiment_bfi()
# experiment.questionnaire.print_questionnaire()

### Load Experiment from a Dictionary

In [3]:
experiment_dict = {
    "name": "Generative Models for Big Five Inventory",
    "description": "Description: Testing Generative Models for BFI questionnaire using Rupsycho.",
    "demographic_profiles": {
        "Profile 1": {
            "attributes": {
                "title": "Mr",
                "name": "Grueber"
            },
            "template": "{title} {name}"
        }
    },
    "questionnaire": None
}

In [4]:
experiment = rup.experiment_from_dict(experiment_dict)

### Set Questionnaires from a Dictionary

In [5]:
experiment_dict = {
    "name": "BIG FIVE INVENTORY RESPONSE FORM AND INSTRUCTIONS TO PARTICIPANTS",
    "general_instruction": "Here are a number of characteristics that may or may not apply to you. For example, do you agree that you are someone who likes to spend time with others? Please return the number corresponding to the answer options to indicate the extent to which you agree or disagree with that statement.",
    "attributes": {
        "dimension": {
            "1": "Extraversion",
            "2": "Agreeableness",
            "3": "Conscientiousness",
            "4": "Neuroticism",
            "5": "Openness"
        }
    },
    "default_answer_options": {
        "1": {
            "text": "1. Disagree strongly",
            "ignored_for_scale": False,
            "weight": 1
        },
        "2": {
            "text": "2. Disagree a little",
            "ignored_for_scale": False,
            "weight": 2
        },
        "3": {
            "text": "3. Neither agree nor disagree",
            "ignored_for_scale": False,
            "weight": 3
        },
        "4": {
            "text": "4. Agree a little",
            "ignored_for_scale": False,
            "weight": 4
        },
        "5": {
            "text": "5. Agree strongly",
            "ignored_for_scale": False,
            "weight": 5
        }
    },
    "instruction_items": [
        {
            "question": "I see myself as someone who...",
            "reversed": False,
            "attributes": {
                "dimension": "1"
            }
        },
        {
            "question": "Tends to find fault with others",
            "reversed": False,
            "attributes": {
                "dimension": "1"
            }
        },
        {
            "question": "Does a thorough job",
            "reversed": False,
            "attributes": {
                "dimension": "1"
            }
        },
        {
            "question": "Is depressed, blue",
            "reversed": False,
            "attributes": {
                "dimension": "1"
            }
        }
    ]
}


### Load Experiment from a File

In [6]:
# Load a single experiment
file_path = "./data/bfi_demo_config.json"
experiment = rup.experiment_from_file(file_path)

# Load multiple experiments using a wildcard
# file_path = "./data/bfi_experiment_demo*.json"
# experiments = rup.experiments_from_files(file_path)
# print("Number of experiments loaded: ", len(experiments))

In [7]:
# Print the questionnaire of the first experiment
experiment.questionnaire.print_questionnaire()

Name: BIG FIVE INVENTORY RESPONSE FORM AND INSTRUCTIONS TO PARTICIPANTS

General Instruction: Here are a number of characteristics that may or may not apply to you. For example, do you agree that you are someone who likes to spend time with others? Please return the number corresponding to the answer options to indicate the extent to which you agree or disagree with that statement.

Demographic Profiles:
- Ms Muller is 18 years old.

Instruction Items:
- Question: I see myself as someone who...
- Question: Tends to find fault with others
- Question: Does a thorough job
- Question: Is depressed, blue


In [8]:
experiment.questionnaire.model_dump()

{'name': 'BIG FIVE INVENTORY RESPONSE FORM AND INSTRUCTIONS TO PARTICIPANTS',
 'general_instruction': 'Here are a number of characteristics that may or may not apply to you. For example, do you agree that you are someone who likes to spend time with others? Please return the number corresponding to the answer options to indicate the extent to which you agree or disagree with that statement.',
 'demographic_profiles': [{'attributes': {'age': 18,
    'title': 'Ms',
    'name': 'Muller'},
   'template': '{title} {name} is {age} years old.'}],
 'attributes': {'dimension': {'1': 'Extraversion',
   '2': 'Agreeableness',
   '3': 'Conscientiousness',
   '4': 'Neuroticism',
   '5': 'Openness'}},
 'default_answer_options': {'1': {'text': '1. Disagree strongly',
   'ignored_for_scale': False,
   'weight': 1},
  '2': {'text': '2. Disagree a little',
   'ignored_for_scale': False,
   'weight': 2},
  '3': {'text': '3. Neither agree nor disagree',
   'ignored_for_scale': False,
   'weight': 3},
  '4'

### Demographic Profiles

In [9]:
print("Number of demographic profiles: ", len(experiment.demographic_profiles.values()))
print("Template:", list(experiment.demographic_profiles.values())[0].template)
print("Attributes:", list(experiment.demographic_profiles.values())[0].attributes.model_dump())
print("Complete Description of the Demo Profile:", list(experiment.demographic_profiles.values())[0])

Number of demographic profiles:  2
Template: {title} {name} is {age} years old and very open minded with a optimistic personality.
Attributes: {'age': 18, 'title': 'Ms', 'name': 'Muller'}
Complete Description of the Demo Profile: Ms Muller is 18 years old and very open minded with a optimistic personality.


## Define Generative Models and Prompts

### Define Parameters

In [10]:
params = {
    "min_new_tokens": 1, 
    "max_new_tokens": 64, 
    "temperature": 0.6, 
    "do_sample": True, 
}

### Load Huggingface Model (local)

In [11]:
from langchain_huggingface.llms import HuggingFacePipeline
from transformers import AutoModelForSeq2SeqLM, AutoModelForCausalLM, AutoTokenizer, pipeline

In [12]:
# #model_id = "HuggingFaceH4/zephyr-7b-beta"#,"lmsys/vicuna-7b-v1.5"#"HuggingFaceTB/SmolLM-1.7b-Instruct"
# #tokenizer = AutoTokenizer.from_pretrained(model_id)
# #model = AutoModelForCausalLM.from_pretrained(model_id)
# #pipe = pipeline("text-generation", model_id, **params, return_full_text = False, device_map = "cuda")
# #model_local = HuggingFacePipeline(pipeline=pipe)

# from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
# from langchain.llms import HuggingFacePipeline

# params = {
#     "min_new_tokens": 1, 
#     "max_new_tokens": 64, 
#     "temperature": 0.6, 
#     "do_sample": True, 
# }

# model_id = "HuggingFaceH4/zephyr-7b-beta"  # Replace with your desired model ID

# # Load the tokenizer
# tokenizer = AutoTokenizer.from_pretrained(model_id)

# # Load the quantized model
# model = AutoModelForCausalLM.from_pretrained(
#     model_id,
#     load_in_4bit=True,  # Load in 4-bit quantization
#     device_map="auto",  # Auto-distribute across available devices (GPU/CPU)
# )

# # Create a pipeline with the quantized model
# pipe = pipeline(
#     "text-generation", 
#     model=model, 
#     tokenizer=tokenizer, 
#     **params,
#     return_full_text=False,
#     #device=0  # Specify the device, 0 for the first GPU, or use device_map for auto allocation
# )

# # Initialize HuggingFacePipeline with the pipeline
# model_local = HuggingFacePipeline(pipeline=pipe)

In [13]:
model_id = "google/flan-t5-small"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForSeq2SeqLM.from_pretrained(model_id)
pipe = pipeline("text2text-generation", model=model, tokenizer=tokenizer, **params)
model_local = HuggingFacePipeline(pipeline=pipe)


/home/jschelb/.pyenv/versions/3.10.8/envs/rup/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [14]:
# prompt_example = "Once upon a time"
# print("Prompt length:", model_local.get_num_tokens(prompt_example))
# print("Prompt Token Ids:", model_local.get_token_ids(prompt_example))
# print("\nAnswer:", model_local.invoke(prompt_example))

### Load Remote Model (Huggingface Endpoint)

In [17]:
# # See https://python.langchain.com/v0.2/docs/integrations/llms/huggingface_endpoint/
# from langchain_huggingface import HuggingFaceEndpoint

# model_remote = HuggingFaceEndpoint(
#     repo_id="HuggingFaceH4/zephyr-7b-beta",
#     task="text-generation",
#     huggingfacehub_api_token = "",
#     **params,
# )

In [18]:
prompt_example = "Once upon a time"
print("Prompt length:", model_local.get_num_tokens(prompt_example))
print("Prompt Token Ids:", model_local.get_token_ids(prompt_example))
print("\nAnswer:", model_local.invoke(prompt_example))

/home/jschelb/.pyenv/versions/3.10.8/envs/rup/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Prompt length: 4
Prompt Token Ids: [7454, 2402, 257, 640]

Answer: Upon a time, we shall see the opulent, wet and sappy, humming with a smile.


### Load Ollama Model

In [19]:
# ollama run gemma:2b

In [20]:
# from langchain_ollama.llms import OllamaLLM

In [21]:
# model_ollama = OllamaLLM(model="gemma2:2b", **params)

In [22]:
# prompt_example = "Once upon a time"
# print("Prompt length:", model_ollama.get_num_tokens(prompt_example))
# print("Prompt Token Ids:", model_ollama.get_token_ids(prompt_example))
# print("\nAnswer:", model_ollama.invoke(prompt_example))

### Load OpenAI Models

In [23]:
# from langchain_openai import ChatOpenAI

# model_openai = ChatOpenAI(
#     model="gpt-4o",
#     temperature=0,
#     max_tokens=None,
#     timeout=None,
#     max_retries=2,
#     api_key="",  # if you prefer to pass api key in directly instaed of using env vars
#     # base_url="...",
#     # organization="...",
#     # other params...
# )

In [24]:
# prompt_example = "Once upon a time"
# print("Prompt length:", model_openai.get_num_tokens(prompt_example))
# print("Prompt Token Ids:", model_openai.get_token_ids(prompt_example))
# print("\nAnswer:", model_openai.invoke(prompt_example))

### Set Prompt Template

In [25]:
from langchain_core.prompts import ChatPromptTemplate

system_message_template = """
    Objective: "{general_instruction}"
    Answer with respect to the following persona description and question.
    """
    
user_message_template= """ 
    Instructions: Choose from the list of answer options to answer the question. Answer the question using only the provided answer options. If none of the options are correct, choose the option that is closest to being correct. Return the number corresponding to the answer option and nothing else!
    
    Question:
    {persona_description} was asked the following question. I see myself as someone who {question}

    Answer Options: 
    {answer_options}
    
    Answer:
    """

chat_prompt = ChatPromptTemplate.from_messages([
    ("system", system_message_template),
    ("user", user_message_template)
])

In [26]:
example_instructions = {
    "general_instruction": "Please answer the following questions with respect to the persona described below.",
    
    "persona_description": "You as a 65 year conservative man work in a coal mine",
    
    "question": "What is your favorite color?",
    "answer_options": " ".join(["Red", "Blue", "Green", "Yellow", "Black", "White", "Brown"])
    
}

print(chat_prompt.invoke(example_instructions))

messages=[SystemMessage(content='\n    Objective: "Please answer the following questions with respect to the persona described below."\n    Answer with respect to the following persona description and question.\n    '), HumanMessage(content=' \n    Instructions: Choose from the list of answer options to answer the question. Answer the question using only the provided answer options. If none of the options are correct, choose the option that is closest to being correct. Return the number corresponding to the answer option and nothing else!\n    \n    Question:\n    You as a 65 year conservative man work in a coal mine was asked the following question. I see myself as someone who What is your favorite color?\n\n    Answer Options: \n    Red Blue Green Yellow Black White Brown\n    \n    Answer:\n    ')]


In [27]:
# from langchain_core.load import dumpd, dumps, load, loads
# dict_representation = dumpd(chat_prompt)
# print(type(dict_representation))
# prompt_2 = load(dict_representation) 
# print(dumps(chat_prompt, pretty=True))

In [28]:
simple_chain = chat_prompt | model_local
answer = simple_chain.invoke(example_instructions)
print("\nAnswer:", answer)


Answer: 4


In [29]:
example_instructions = {
    "general_instruction": "Please answer the following questions with respect to the persona described below.",
    
    "persona_description": "You as 18 year old student living in the city at the sea side, liking to go to the beach and party with friends, looking into the sky and dreaming about the future",
    
    "question": "What is your favorite color?",
    "answer_options": " ".join(["Red", "Blue", "Green", "Yellow"])
    
}

print(chat_prompt.invoke(example_instructions))

messages=[SystemMessage(content='\n    Objective: "Please answer the following questions with respect to the persona described below."\n    Answer with respect to the following persona description and question.\n    '), HumanMessage(content=' \n    Instructions: Choose from the list of answer options to answer the question. Answer the question using only the provided answer options. If none of the options are correct, choose the option that is closest to being correct. Return the number corresponding to the answer option and nothing else!\n    \n    Question:\n    You as 18 year old student living in the city at the sea side, liking to go to the beach and party with friends, looking into the sky and dreaming about the future was asked the following question. I see myself as someone who What is your favorite color?\n\n    Answer Options: \n    Red Blue Green Yellow\n    \n    Answer:\n    ')]


In [30]:
answer = simple_chain.invoke(example_instructions)
print("\nAnswer:", answer)


Answer: 5


### Define Output Parser

In [31]:
from rupsycho.parser import BasicParser 
basic_parser = BasicParser()

In [32]:
simple_chain = chat_prompt | model_local | basic_parser
answer = simple_chain.invoke(example_instructions)
print("\nAnswer:", answer)


Answer: 3


## Run Experiment

### Single Experiment

In [33]:
experiment.models

{'default_model': LocalHuggingFaceModelConfig(type='local_huggingface', hf_model_name_or_path='HuggingFaceTB/SmolLM-135M-Instruct', revision=None, tokenizer_name=None, cache_dir=None, use_auth_token=False, device='cpu', pipeline_type='text-generation')}

In [34]:
## Register Models
#experiment.add_model(model_local)
experiment.add_model(model_local)

# Register the prompt template
experiment.set_prompt(chat_prompt)

# Set the parser
experiment.set_parser(basic_parser)

In [35]:
experiment.list_models()

['default_model', '140223432922624']

In [36]:
# Test the chain with first model in the list
runnable_chain = experiment.create_chain(model_key=experiment.list_models()[0])
runnable_chain.invoke(example_instructions)

'System: Objective: "Please answer the following questions with respect to the persona described below." Answer with respect to the following persona description and question. Human: Instructions: Choose from the list of answer options to answer the question. Answer the question using only the provided answer options. If none of the options are correct, choose the option that is closest to being correct. Return the number corresponding to the answer option and nothing else! Question: You as 18 year old student living in the city at the sea side, liking to go to the beach and party with friends, looking into the sky and dreaming about the future was asked the following question. I see myself as someone who What is your favorite color? Answer Options: Red Blue Green Yellow Answer: Human: Instructions: Choose from the list of answer options to answer the question. Answer the question using only the provided answer options. If none of the options are correct, choose the option'

In [37]:
# Run the experiment
experiment.run()

Generative Models for Big Five Inventory:   0%|          | 0/16 [00:00<?, ? prompts/s]

In [38]:
from langchain_core.output_parsers import StrOutputParser
parser = StrOutputParser()
parser.invoke(answer)

from rupsycho.parser import BasicParser 
basic_parser = BasicParser()
basic_parser.invoke(answer)

'3'

In [39]:
# from langchain_core.load import dumpd, dumps, load, loads
# runnable_object = basic_parser
# dict_representation = dumpd(runnable_object)
# print(type(dict_representation))
# restored_object = load(dict_representation) 
# print(dumps(runnable_object, pretty=True))

### Multiple Experiments

In [40]:
from rupsycho.experiment_collection import ExperimentCollection

experiment_collection = ExperimentCollection([experiment, experiment])

print("Experiments in collection:", len(experiment_collection))

## Set the prompt for all experiments
# experiment_collection.set_prompt(chat_prompt)

## Set the models for all experiments
# experiment_collection.add_model(model_local)

## Run multiple experiments
#experiment_collection.run_all()

Experiments in collection: 2


## Save Results

In [41]:
## Export Resuts
experiment.get_answers()[:5]

/home/jschelb/.pyenv/versions/3.10.8/envs/rup/lib/python3.10/site-packages/pydantic/main.py:364: UserWarning: Pydantic serializer warnings:
  Expected `int` but got `str` - serialized value may not be as expected
  Expected `int` but got `str` - serialized value may not be as expected
  Expected `int` but got `str` - serialized value may not be as expected
  Expected `int` but got `str` - serialized value may not be as expected
  Expected `int` but got `str` - serialized value may not be as expected
  Expected `int` but got `str` - serialized value may not be as expected
  Expected `int` but got `str` - serialized value may not be as expected
  Expected `int` but got `str` - serialized value may not be as expected
  Expected `int` but got `str` - serialized value may not be as expected
  Expected `int` but got `str` - serialized value may not be as expected
  return self.__pydantic_serializer__.to_python(


[{'default_model': {'Optimistic Persona': {'7': 'System: Objective: "Here are a number of characteristics that may or may not apply to you. For example, do you agree that you are someone who likes to spend time with others? Please return the number corresponding to the answer options to indicate the extent to which you agree or disagree with that statement." Answer with respect to the following persona description and question. Human: Instructions: Choose from the list of answer options to answer the question. Answer the question using only the provided answer options. If none of the options are correct, choose the option that is closest to being correct. Return the number corresponding to the answer option and nothing else! Question: Ms Muller is 18 years old and very open minded with a optimistic personality. was asked the following question. I see myself as someone who I see myself as someone who... Answer Options: 1. Disagree strongly 2. Disagree a little 3. Neither agree nor disag

In [42]:
experiment.questionnaire.instruction_items[0]

InstructionItem(question='I see myself as someone who...', reversed=False, answer_options=None, attributes={'dimension': '1'}, answers=defaultdict(<function InstructionItem.<lambda>.<locals>.<lambda> at 0x7f88432d4670>, {'default_model': defaultdict(<function InstructionItem.<lambda>.<locals>.<lambda>.<locals>.<lambda> at 0x7f8838e8ed40>, {'Optimistic Persona': defaultdict(<class 'dict'>, {'7': 'System: Objective: "Here are a number of characteristics that may or may not apply to you. For example, do you agree that you are someone who likes to spend time with others? Please return the number corresponding to the answer options to indicate the extent to which you agree or disagree with that statement." Answer with respect to the following persona description and question. Human: Instructions: Choose from the list of answer options to answer the question. Answer the question using only the provided answer options. If none of the options are correct, choose the option that is closest to b

In [43]:
experiment.get_answers_as_dataframe().head(100)

,Instruction ID,Instruction Question,Model ID,Persona ID,Run Seed,Answer
0,0,I see myself as someone who...,default_model,Optimistic Persona,7,"System: Objective: ""Here are a number of chara..."
1,0,I see myself as someone who...,default_model,Conservative Persona,7,"System: Objective: ""Here are a number of chara..."
2,0,I see myself as someone who...,140223432922624,Optimistic Persona,7,5.
3,0,I see myself as someone who...,140223432922624,Conservative Persona,7,5.
4,1,Tends to find fault with others,default_model,Optimistic Persona,7,"System: Objective: ""Here are a number of chara..."
5,1,Tends to find fault with others,default_model,Conservative Persona,7,"System: Objective: ""Here are a number of chara..."
6,1,Tends to find fault with others,140223432922624,Optimistic Persona,7,5
7,1,Tends to find fault with others,140223432922624,Conservative Persona,7,5.
8,2,Does a thorough job,default_model,Optimistic Persona,7,"System: Objective: ""Here are a number of chara..."
9,2,Does a thorough job,default_model,Conservative Persona,7,"System: Objective: ""Here are a number of chara..."


In [45]:
# Write the results to a file
file_path_out = "data/bfi_experiment_results_demo_2.json"
experiment.export_to_file(file_path_out)

/home/jschelb/.pyenv/versions/3.10.8/envs/rup/lib/python3.10/site-packages/pydantic/main.py:364: UserWarning: Pydantic serializer warnings:
  Expected `int` but got `str` - serialized value may not be as expected
  Expected `int` but got `str` - serialized value may not be as expected
  Expected `int` but got `str` - serialized value may not be as expected
  Expected `int` but got `str` - serialized value may not be as expected
  Expected `int` but got `str` - serialized value may not be as expected
  Expected `int` but got `str` - serialized value may not be as expected
  Expected `int` but got `str` - serialized value may not be as expected
  Expected `int` but got `str` - serialized value may not be as expected
  Expected `int` but got `str` - serialized value may not be as expected
  Expected `int` but got `str` - serialized value may not be as expected
  Expected `int` but got `str` - serialized value may not be as expected
  Expected `int` but got `str` - serialized value may not